# Tarea 4: Restauración de imagenes

## Instrucciones

* La tarea es individual.
* Las consultas sobre las tareas se deben realizar por medio de la plataforma Aula.
* La tarea debe ser realizada en `Jupyter Notebook` (`Python3`).
* Se evaluará la correcta utilización de librerias `NumPy`, `SciPy`, entre otras, así como la correcta utilizacion de algoritmos de forma vectorizada.
*  **El archivo de entrega debe denominarse ROL-tarea-numero.ipynb**. _De no respetarse este formato existirá un descuento de **50 puntos**_
* La fecha de entrega es el viernes 31 de Julio a las **18:00 hrs**.  Se aceptarán entregas hasta las 19:00 hrs sin descuento en caso de existir algun problema, posteriormente existirá un descuento lineal hasta las 20:00 hrs del mismo día.
* Las tareas que sean entregadas antes del jueves a mediodía recibirán una bonificación de 10 puntos
* Debe citar cualquier código ajeno utilizado (incluso si proviene de los Jupyter Notebooks del curso).

In [ ]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from skimage.metrics import structural_similarity as ssim
from scipy.optimize import minimize

# Introducción

Como se vio en la tarea anterior la interpolación bicúbica sirve para aumentar la dimensión de una imagen obteniendo valores de nuevos pixeles interiores. En esta tarea se verá otra aplicación a la interpolación bicúbica para la restauración de imágenes

# Eliminación de ruido 


Supongamos que se tiene una imagen $X$ cuyos pixeles presentan valores $I_{xy}$ con ruido y se quiere volver a la imagen original


In [ ]:

from skimage import data, img_as_float
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import mean_squared_error
from skimage import io
from skimage import color

img = color.rgb2gray(io.imread('test.png'))

img = img_as_float(img)

rows, cols = img.shape


noise = np.ones_like(img) * 0.2 * (img.max() - img.min())
noise[np.random.random(size=noise.shape) > 0.5] *= -1

img_noise = img + noise

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 4),
                         sharex=True, sharey=True)
ax = axes.ravel()


ax[0].imshow(img, cmap=plt.cm.gray, vmin=0, vmax=1)

ax[0].set_title('Original image')

ax[1].imshow(img_noise, cmap=plt.cm.gray, vmin=0, vmax=1)
ax[1].set_title('Image with noise')

plt.tight_layout()
plt.show()

El método a implementar, será utilizando la interpolación bicúbica que viene representada por la siguiente spline

$$
p(x,y) = \sum\limits_{i=0}^3 \sum_{j=0}^3 a_{ij} x^i y^j.
$$
Como se puede observar el problema de interpolación se resume en determinar los 16 coeficientes $a_{ij}$. Ya que la imagen presenta ruido, no se puede despejar los coeficientes asumiendo que los valores de las derivadas $f_x$, $f_y$ y $f_{xy}$ son correctas. 

Para determinar los coeficientes se seleccionara secciones solapadas de la imagen de $5\times5 $ pixeles y se determinará una función $p_i$ por cada sección, ejemplificando con una imagen de $9\times 9$  las secciones quedarían de la siguiente manera:


<img src="matriz_seccion.png" width="50%"/>

Se busca la minimización del error cuadrático entre la spline generada $p_i$ y el punto de la imagen, en el caso del ejemplo anterior la función objetivo a minimizar es:

$$(p_1(0,0)-I_{00})^2 + (p_1(0,1)-I_{01})^2 +\dots+(p_1(4,4)-I_{44})^2 + (p_2(4,4)-I_{44})^2+ \dots+(p_4(8,8)-I_{88})^2 $$


Ya que la imagen debe presentar continuidad entre las secciones los pixeles que abarcan más de una deben presentar el mismo valor por lo tanto existen restricciones que se deben cumplir:

$$p_1(0,4) = p_2(0,4)$$
$$p_1(4,4) = p_2(4,4)$$
$$p_1(4,0) = p_3(4,0)$$
$$p_1(4,4) = p_3(4,4)$$
$$\vdots$$

Es decir los valores de los pixeles de solamente las **esquinas** deben ser iguales en todas las secciones que coinciden 

Finalmente al obtener los valores de los coeficientes la nueva imagen será el resultado de la evaluación de todos los pixeles en su spline respectiva

# Preguntas

## 1.Restauración de imagenes

### 1.1 Generar $p_i(x,y)$ (10 pts)

Debe implementar la función `spline_evaluate` que reciba un arreglo con los valores de los coeficientes y el valor de la coordenada $x$ e $y$ del pixel y debe retornar el valor del pixel evaluado en la spline


In [ ]:
def spline_evaluate(a, pos):
    '''
    a: (array 16) arreglo de coeficientes
    pos: (tuple) tupla con forma (x,y) que representa la posicion del pixel
    
    return
    value: (float) evaluacion del pixel
      
    '''
    new_a = a.reshape(4,4).T
    
    x, y = pos
    P = lambda x, y, a: np.array([x**i for i in range(4)]) @ a @ np.array([y**i for i in range(4)])
    
    value = P(x, y, new_a)
    
    return value

### 1.2 Generar funcion a minimizar (25 pts)

Debe implementar la función `objective_function` que reciba un arreglo con los valores de todos los coeficientes necesarios y la imagen con ruido, y debe retornar el error cuadrático entre el polinomio y el valor del pixel de la imagen.


In [ ]:
def objective_function(a,image):
    '''
    a: (array) array con todos los coeficientes 
    image: (array nxn) imagen que presenta ruido en sus datos
    
    return
    error: suma total del error cuadratico entre la spline evaluada y el valor del pixel respectivo
    
    '''
    
    n = image.shape[0]
    
    error = 0
    
    # function to compute sqrd error
    errSqrd = lambda p_sec, I_sec: np.sum([(p_ij - I_ij)**2 for p_ij, I_ij in zip(p_sec, I_sec)])
    
    # index for array of alphas
    a_index = 0
    # n = 4k + 1, for some k   
    for i in range(0, n, 4):
        for j in range(0, n, 4):
            if i+4 < n and j + 4 < n:

                # sqr pos of matrix
                I_s = (image[i: i+5, j: j+5])

                # this is big brain stuff
                # square matrix (5x5) with i,j coefficients in each pos
                # eg: if i = 1 then p_ij[0] = [1,1]; p_ij[1] = [1,2]; etc
                # p_xs = np.mgrid[i:i+5, j:j+5].reshape(2,-1).T
                p_xs = np.mgrid[0:1.25:0.25, 0:1.25:0.25].reshape(2,-1).T

                # slice a to form matrix for MATRIX MULTIPLICATION
                a_secc = a[a_index : a_index + 16]

                # hasta ahora esta sumando de mas las partes p_1(x,y) = p_2(x,y)

                # compute sline evaluate for every pixel in p_xs
                p_s = np.array([spline_evaluate(a_secc, p) for p in p_xs])

                # compute squared difference between pixel and p(i,j)
                error += errSqrd(p_s, I_s.reshape(25))

                a_index += 16
                
    return error

### 1.3 Generar Restricciones (25 pts)

Se debe implementar la función `create_constraints` que reciba la imagen y retorne una lista de diccionarios con las restricciones del problema. El diccionario debe tener la siguiente estructura:

`{"type: "eq", "fun": funcion_con_restriccion}`



In [ ]:
def create_constraints(image):
    '''
    image: (array nxn) imagen que presenta ruido en sus datos

    return
    constraints: (list) lista de restricciones  
    '''
    constraints = list()
    
    n = image.shape[0]
    n_splines = 0
    
    # ROW CONSTRAINTS
    for row_up_down in range(4, n-1, 4):
        
        # UP
        ru = row_up_down*16
        ru_next = (row_up_down+1)*16
        
        constranints.append( {"type":"eq", "fun": lambda a, row_up_down: 
                spline_evaluate(a[ru: ru+16], (0,1)) == spline_evaluate(a[ru_next: ru_next+16], (0,0))})
        
        # DOWN
        rd = (row_up_down + n_splines)*16
        rd_next = ((row_up_down +n_splines)+1)*16
        constranints.append( {"type":"eq", "fun": lambda a, row_up_down: 
                spline_evaluate(a[rd: r_u_drd+16], (1,1)) == spline_evaluate(a[rd: r_u_d+16], (1,0))})
        
        n_splines +=1
    
    # COL CONSTRAINTS
    for row_up_down in range(4, n-1, 4):
        r_u_d = row_up_down*16
        # LEFT
        constranints.append( {"type":"eq", "fun": lambda a, row_up_down: 
                spline_evaluate(a[r_u_d: r_u_d+16], (1,0)) == spline_evaluate(a[r_u_d: r_u_d+16], (0,0))})
        # RIGHT
        constranints.append( {"type":"eq", "fun": lambda a, row_up_down: 
                spline_evaluate(a[r_u_d: r_u_d+16], (1,1)) == spline_evaluate(a[r_u_d: r_u_d+16], (0,1))})
    
    
    for i in range(0, n, 4):
        for j in range(0, n, 4):
            if i+4 < n and j + 4 < n:
                """ xd """
                
            
            
    
    
    return constraints

### 1.4 Generar nueva imagen (30 pts)
Se debe implementar la función `clean_image` que reciba un arreglo con una imagen en escala de grises y si es que se consideran las restricciones de continuidad. Debe retornar una nueva imagen con el ruido minimizado. 
Para realizar la minimización, debe utilizarse la función [minimize de la libreria de scipy](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html)


In [ ]:
def clean_image(image,constraints):
    '''
    image: (array nxn) imagen con ruido
    constraints: (bool) true si es que se consideran las restricciones
    
    return
    new_iamge: (array nxn) imagen restaurada
    '''
    
    n = image.shape[0]
    x0 = np.zeros(int(16*((n-5)/4 + 1)**2))
    
    if constraints:
        cons = create_constraints(image)
        alphas = sp.optimize.minimize(objective_function, x0,(image), constraints = cons)
    else:
        alphas = sp.optimize.minimize(objective_function, x0,(image))
    print(alphas)
    
    a = alphas.x
    # prepare new_image
    new_image = np.zeros_like(image)
    
    splines = (a.shape[0]/16)
    
    # offset in new images
    new_r = 0
    new_l = 0
    
    for s in range(int(splines)):
        for row_m in range(0,5):
            for col_m in range(0,5):
                new_image[row_m+new_r , col_m + new_l] = spline_evaluate(
                                                                        a[s*16: s*16 +16], (row_m/4, col_m/4))
        new_r += 4
        if(new_r >= n-1):
            new_r=0
            new_l +=4    
        
    return new_image

In [ ]:
img_new = clean_image(img_noise, False)
imgplot = plt.imshow(img_new, cmap=plt.cm.gray)
plt.show()

## 2. Evaluar error (5 pts)

Implemente la función `error_restore` la cual debe obtener el error de la imagen obtenida comparándola con una de referencia. El error debe ser calculado utilizando el índice SSIM (Structural similarity)

In [ ]:
def error_restore(original,new):
    """
    Parameters
    ----------
    image:	(nxn array) imagen original limpia
    new:	(nxn array) imagen restaurada


    Returns
    -------
    error:	(float) diferencia entre imagenes (1-ssim) 

    """
    n = new.shape[0]
    error = 1 - ssim(original, new, multichannel=True, data_shape=n)
    
    return error

**Pregunta: ¿Como afecta el uso de restricciones en el error?** (5 pts)

Al hacer uso de constraints, el error debería disminuir. Esto debido a que se fuerza al sistema a tener puntos de continuidad, resultando en una mayor nitidez en la imagen.
Además, con las restricciones se hace un tradeoff entre calidad de la imagen y tiempo de ejecución.

# Consideraciones

* Se trabajará con imágenes cuadradas en escala de grises
* Las imágenes tendrán una dimensión adecuada para que no sobren o falten pixeles para la agrupación de $5\times5$
* Para los casos de los pixeles que no tienen restricción pero pueden ser evaluados por distintas splines considere el valor de la spline de la izquierda o superior
